# Bibliotecas 

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Carregar a base de dados do arquivo CSV


In [2]:

url = "https://github.com/valdineyatilio/ProjetoAplicado-III/raw/refs/heads/main/Aula-02/BaseDeDados-AmazonProductReviews.csv"
dados_Amazon = pd.read_csv(url, encoding='latin1', sep=';')


# Preencher valores nulos na coluna 'Text' com strings vazias


In [4]:
dados_Amazon['Text'] = dados_Amazon['Text'].fillna('')


# Aplicar TF-IDF para transformar os textos em vetores numéricos


In [5]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=2, ngram_range=(1,2))
matriz_tfidf = tfidf.fit_transform(dados_Amazon['Text'])



# Calcular a similaridade entre os produtos


In [6]:
similaridade = cosine_similarity(matriz_tfidf)

# Função para avaliar precisão da similaridade


In [7]:
def avaliar_precisao(n_amostras=10):
    precisao_total = 0
    for i in np.random.choice(len(dados_Amazon), n_amostras, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        
        score_medio = np.mean([score for _, score in similares[1:6]])  # Média dos top 5 similares
        precisao_total += score_medio
    
    return precisao_total / n_amostras



# Função para medir cobertura dos produtos

In [8]:
def avaliar_cobertura():
    produtos_recomendados = set()
    for i in np.random.choice(len(dados_Amazon), 100, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        produtos_recomendados.update([dados_Amazon.iloc[idx]['ProductId'] for idx, _ in similares[1:6]])
    
    return len(produtos_recomendados) / len(dados_Amazon['ProductId'].unique())



# Função para avaliar diversidade


In [10]:
def avaliar_diversidade():
    diversidade_total = 0
    for i in np.random.choice(len(dados_Amazon), 10, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        categorias = [dados_Amazon.iloc[idx]['Summary'] for idx, _ in similares[1:6]]
        diversidade_total += len(set(categorias)) / len(categorias)
    
    return diversidade_total / 10


# Teste das métricas


In [11]:
print(f"\nPrecisão média da similaridade: {avaliar_precisao():.4f}")
print(f"Cobertura dos produtos recomendados: {avaliar_cobertura():.4f}")
print(f"Diversidade das recomendações: {avaliar_diversidade():.4f}")



Precisão média da similaridade: 0.3511
Cobertura dos produtos recomendados: 0.2903
Diversidade das recomendações: 1.0000
